In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

#### Loading the data

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### Network class

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

#### Training the network with SGD

In [8]:
# Logging for tensorboard
writer = SummaryWriter()

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train (SGD)", loss, epoch*len(trainloader)+i)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
model_dict = model.state_dict()
writer.flush()
writer.close()

[1,  2000] loss: 2.307
[1,  4000] loss: 2.305
[1,  6000] loss: 2.301
[1,  8000] loss: 2.299
[1, 10000] loss: 2.296
[1, 12000] loss: 2.293
[2,  2000] loss: 2.288
[2,  4000] loss: 2.285
[2,  6000] loss: 2.279
[2,  8000] loss: 2.272
[2, 10000] loss: 2.264
[2, 12000] loss: 2.255
[3,  2000] loss: 2.237
[3,  4000] loss: 2.227
[3,  6000] loss: 2.210
[3,  8000] loss: 2.201
[3, 10000] loss: 2.181
[3, 12000] loss: 2.168
[4,  2000] loss: 2.153
[4,  4000] loss: 2.138
[4,  6000] loss: 2.133
[4,  8000] loss: 2.111
[4, 10000] loss: 2.103
[4, 12000] loss: 2.092
[5,  2000] loss: 2.072
[5,  4000] loss: 2.063
[5,  6000] loss: 2.049
[5,  8000] loss: 2.039
[5, 10000] loss: 2.032
[5, 12000] loss: 2.035
[6,  2000] loss: 2.018
[6,  4000] loss: 2.006
[6,  6000] loss: 2.001
[6,  8000] loss: 1.999
[6, 10000] loss: 1.991
[6, 12000] loss: 1.982
[7,  2000] loss: 1.980
[7,  4000] loss: 1.974
[7,  6000] loss: 1.973
[7,  8000] loss: 1.963
[7, 10000] loss: 1.962
[7, 12000] loss: 1.965
[8,  2000] loss: 1.965
[8,  4000] 

#### Testing the network with SGD

In [9]:
dataiter = iter(testloader)
images, labels = next(dataiter)

model2 = Net()
model2.load_state_dict(model_dict)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
# Learning off
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model2(images)
        _, preds = torch.max(outputs.data, 1)
        for label, pred in zip(labels, preds):
            if label == pred:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

accs = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    accs.append(accuracy)
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
avg_acc = sum(accs) / len(accs)
print(f'Total accuracy: {avg_acc:.1f} %')

Accuracy for class: plane is 45.2 %
Accuracy for class: car   is 41.8 %
Accuracy for class: bird  is 3.6 %
Accuracy for class: cat   is 7.9 %
Accuracy for class: deer  is 7.0 %
Accuracy for class: dog   is 35.3 %
Accuracy for class: frog  is 64.2 %
Accuracy for class: horse is 41.8 %
Accuracy for class: ship  is 41.1 %
Accuracy for class: truck is 42.1 %
Total accuracy: 33.0 %


Using SGD as the optimizer and Tanh as the activation function, the model achieved 33% testing accuracy.

#### Training the network with Adam

In [10]:
# Logging for tensorboard
writer = SummaryWriter()

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/train (Adam)", loss, epoch*len(trainloader)+i)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')
model_dict = model.state_dict()
writer.flush()
writer.close()

[1,  2000] loss: 1.979
[1,  4000] loss: 1.759
[1,  6000] loss: 1.680
[1,  8000] loss: 1.642
[1, 10000] loss: 1.606
[1, 12000] loss: 1.537
[2,  2000] loss: 1.487
[2,  4000] loss: 1.463
[2,  6000] loss: 1.433
[2,  8000] loss: 1.414
[2, 10000] loss: 1.418
[2, 12000] loss: 1.394
[3,  2000] loss: 1.339
[3,  4000] loss: 1.340
[3,  6000] loss: 1.354
[3,  8000] loss: 1.317
[3, 10000] loss: 1.331
[3, 12000] loss: 1.315
[4,  2000] loss: 1.262
[4,  4000] loss: 1.259
[4,  6000] loss: 1.263
[4,  8000] loss: 1.271
[4, 10000] loss: 1.241
[4, 12000] loss: 1.249
[5,  2000] loss: 1.197
[5,  4000] loss: 1.219
[5,  6000] loss: 1.213
[5,  8000] loss: 1.193
[5, 10000] loss: 1.177
[5, 12000] loss: 1.179
[6,  2000] loss: 1.145
[6,  4000] loss: 1.154
[6,  6000] loss: 1.178
[6,  8000] loss: 1.142
[6, 10000] loss: 1.132
[6, 12000] loss: 1.126
[7,  2000] loss: 1.099
[7,  4000] loss: 1.110
[7,  6000] loss: 1.110
[7,  8000] loss: 1.096
[7, 10000] loss: 1.110
[7, 12000] loss: 1.099
[8,  2000] loss: 1.064
[8,  4000] 

In [11]:
#### Testing the network with Adam

In [12]:
dataiter = iter(testloader)
images, labels = next(dataiter)

model2 = Net()
model2.load_state_dict(model_dict)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
# Learning off
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model2(images)
        _, preds = torch.max(outputs.data, 1)
        for label, pred in zip(labels, preds):
            if label == pred:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

accs = []
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    accs.append(accuracy)
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
avg_acc = sum(accs) / len(accs)
print(f'Total accuracy: {avg_acc:.1f} %')

Accuracy for class: plane is 69.4 %
Accuracy for class: car   is 71.7 %
Accuracy for class: bird  is 43.4 %
Accuracy for class: cat   is 42.0 %
Accuracy for class: deer  is 52.1 %
Accuracy for class: dog   is 58.7 %
Accuracy for class: frog  is 74.7 %
Accuracy for class: horse is 66.1 %
Accuracy for class: ship  is 74.1 %
Accuracy for class: truck is 72.3 %
Total accuracy: 62.5 %


Using Adam as the optimizer and Tanh as the activation function, the model achieved 62.5% testing accuracy.